# 상권 코드 좌표 변환
- 상권 코드 : EPSG: 5181 사용
- 변환할 좌표계: EPSG: 4326 (학교 위치 데이터에서 사용)

In [32]:
# 라이브러리
from pyproj import Proj, Transformer, CRS
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [33]:
# 상권 코드 좌표 데이터

df = pd.read_csv("/Users/jiwon/portfolio/school-location-based-market-data-analysis/datasets/raw-datasets/seoul-region.csv")
df1 = df.copy()
# necessary columns
region = df[["상권_코드", "상권_코드_명", "엑스좌표_값", "와이좌표_값"]]
region.head(3)

,상권_코드,상권_코드_명,엑스좌표_값,와이좌표_값
0,3110008,배화여자대학교(박노수미술관),197093,453418
1,3110009,자하문터널,196991,455057
2,3110010,평창동서측,197064,456643


In [35]:
# EPSg: 5181
crs_5181

<Geographic 2D CRS: EPSG:4258>
Name: ETRS89
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: Europe - onshore and offshore: Albania; Andorra; Austria; Belgium; Bosnia and Herzegovina; Bulgaria; Croatia; Cyprus; Czechia; Denmark; Estonia; Faroe Islands; Finland; France; Germany; Gibraltar; Greece; Hungary; Ireland; Italy; Kosovo; Latvia; Liechtenstein; Lithuania; Luxembourg; Malta; Moldova; Monaco; Montenegro; Netherlands; North Macedonia; Norway including Svalbard and Jan Mayen; Poland; Portugal; Romania; San Marino; Serbia; Slovakia; Slovenia; Spain; Sweden; Switzerland; United Kingdom (UK) including Channel Islands and Isle of Man; Vatican City State.
- bounds: (-16.1, 32.88, 40.18, 84.73)
Datum: European Terrestrial Reference System 1989 ensemble
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [36]:
# EPSG: 4326
crs_4326

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [43]:
# 프로젝션 정의
'''
- 중부원점(GRS80)-falseY:50000: 서울시 열린데이터광장에서 제공하는 데이터에 사용된 좌표계 
- EPSG:5181
- 서울시 상권분석서비스(상권-영역)에 사용된 좌표계
'''
proj_1 = Proj(init='epsg:5181')


'''
- WGS84 경위도 
- EPSG:4326
- 학구도안내서비스에서 제공하는 학교 위치 데이터에서 사용
'''
proj_2 = Proj(init='epsg:4326')

# 변환환 좌표를 경위도로 저장
converted = transform(proj_1, proj_2, df['엑스좌표_값'].values, df['와이좌표_값'].values)
df['경도'] = converted[0]
df['위도'] = converted[1]

In [45]:
df['경도']

0       126.967090
1       126.965928
2       126.966748
3       126.983935
4       126.987354
           ...    
1645    126.981856
1646    127.011274
1647    126.997691
1648    127.115241
1649    127.059743
Name: 경도, Length: 1650, dtype: float64

In [46]:
df['위도']

0       37.580309
1       37.595076
2       37.609366
3       37.583584
4       37.582764
          ...    
1645    37.564059
1646    37.567321
1647    37.570007
1648    37.516431
1649    37.511084
Name: 위도, Length: 1650, dtype: float64

In [48]:
# 변환된 경도, 위도 좌표가 데이터프레임에 추가된 것을 확인
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1650 entries, 0 to 1649
Data columns (total 15 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   상권_구분_코드    1650 non-null   object 
 1   상권_구분_코드_명  1650 non-null   object 
 2   상권_코드       1650 non-null   int64  
 3   상권_코드_명     1650 non-null   object 
 4   엑스좌표_값      1650 non-null   int64  
 5   와이좌표_값      1650 non-null   int64  
 6   자치구_코드      1650 non-null   int64  
 7   자치구_코드_명    1650 non-null   object 
 8   행정동_코드      1650 non-null   int64  
 9   행정동_코드_명    1650 non-null   object 
 10  영역_면적       1650 non-null   int64  
 11  lon         1650 non-null   float64
 12  lat         1650 non-null   float64
 13  경도          1650 non-null   float64
 14  위도          1650 non-null   float64
dtypes: float64(4), int64(6), object(5)
memory usage: 193.5+ KB


In [50]:
# 파일 저장
df.to_csv("market-region-converted.csv")